In [8]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('chessboard.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [9]:
# Define the chessboard size
chessboard_size = (7, 7)  # Number of inner corners per a chessboard row and column

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

if ret:
    # Draw and display the corners
    cv2.drawChessboardCorners(image, chessboard_size, corners, ret)
    cv2.imshow('Chessboard Corners', image)
    cv2.waitKey(0)
else:
    print("Chessboard corners not found.")


Chessboard corners not found.


In [7]:
# Define the destination points for the perspective transform
dst_points = np.array([[0, 0], [800, 0], [800, 800], [0, 800]], dtype='float32')

# Perform the perspective transformation
M = cv2.getPerspectiveTransform(corners, dst_points)
warped = cv2.warpPerspective(image, M, (800, 800))

cv2.imshow('Warped Chessboard', warped)
cv2.waitKey(0)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\imgwarp.cpp:3391: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'cv::getPerspectiveTransform'


In [ ]:
square_size = 100  # Size of each square in pixels

squares = []
for row in range(8):
    for col in range(8):
        x = col * square_size
        y = row * square_size
        square = warped[y:y + square_size, x:x + square_size]
        squares.append(square)
        # Optionally display each square
        cv2.imshow(f'Square {row},{col}', square)
        cv2.waitKey(100)